# Working with H3 data

H3 is a popular icosahedral DGGS with hexagonal cells, developed and popularized by Uber. For more information, see https://h3geo.org. The tutorial aims to showcase how to work with H3 data using `xdggs`.

## Import libraries

In [ ]:
import xarray as xr

import xdggs

_ = xr.set_options(display_expand_data=False)

## Initialization

To initialize, we first have to open the dataset. Here we'll use `xarray`'s `air_temperature` tutorial dataset, which was interpolated to the H3 grid.

```{tip}
If the dataset you want to work on is not already on a H3 grid, you will have to use a different package to interpolate.
```

```{warning}
For the purpose of this tutorial we drop the geographic coordinates and load all data into memory, but this is not required.
```

In [ ]:
original_ds = xdggs.tutorial.open_dataset("air_temperature", "h3").load()
air_temperature = original_ds.drop_vars(["lat", "lon"])
air_temperature

After that, we can use {py:func}`xdggs.decode` to tell `xdggs` to interpret the cell ids.

This will create a grid object (see {py:attr}`xarray.Dataset.dggs.grid_info` and {py:class}`xdggs.H3Info` for more information) containing the grid parameters and a custom index for the `cell_ids` coordinate (notice how the coordinate name is displayed in bold), which will allow us to perform grid-aware operations.

````{important}
For this to work, the dataset has to have a coordinate called `cell_ids`, and it also has to have the `grid_name` and `level` attributes.

The `grid_name` refers to the short name of the grid, while `level` refers to the grid hierarchical level (the `h3` libraries call this the "resolution", while `xdggs` will use "level" for all grids).

In this case, the attributes on `cell_ids` are:
```python
{
    "grid_name": "h3",
    "level": 2,
}
```
````

In [ ]:
ds = air_temperature.pipe(xdggs.decode)
ds

## Deriving data

With the grid object and the custom index, we can derive additional data from the cell ids.

### Cell center coordinates

For example, we can reconstruct the cell centers we dropped from the original dataset, using {py:meth}`xarray.Dataset.dggs.cell_centers`:

In [ ]:
cell_centers = ds.dggs.cell_centers()
cell_centers

These are the same as the ones we dropped before:

In [ ]:
derived_ds = ds.assign_coords(
    cell_centers.rename_vars({"latitude": "lat", "longitude": "lon"}).coords
)
derived_ds

In [ ]:
xr.testing.assert_allclose(derived_ds, original_ds)

### Cell boundary polygons

Additionally, we can derive the cell boundary polygons as an array of {doc}`shapely:index` using {py:meth}`xarray.Dataset.dggs.cell_boundaries`:

In [ ]:
cell_boundaries = ds.dggs.cell_boundaries()
cell_boundaries

## Plotting

We can quickly visualize the data using {py:meth}`xarray.DataArray.dggs.explore`, which is powered by [lonboard](https://github.com/developmentseed/lonboard).

```{warning}
This is currently restricted to 1D `DataArray` objects, so we need to select a single timestep.
```

In [ ]:
ds["air"].isel(time=15).dggs.explore()